In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, timedelta

# ----------------------------
# PARAMETERS
# ----------------------------
SYMBOL = "BTCUSDc"
TIMEFRAME = mt5.TIMEFRAME_D1
LOOKBACK_DAYS = 365 * 7  # 7 years
SEQ_LENGTH = 60
FEATURES = ['Close', 'pctB', 'RSI', 'MACD', 'Signal_Line', 'Momentum', 'ATR']

# ----------------------------
# HELPER FUNCTIONS
# ----------------------------
def initialize_mt5():
    if not mt5.initialize():
        raise RuntimeError(f"Failed to initialize MT5: {mt5.last_error()}")

def fetch_data(symbol, timeframe, lookback_days):
    utc_now = datetime.utcnow()
    from_date = utc_now - timedelta(days=lookback_days)
    rates = mt5.copy_rates_range(symbol, timeframe, from_date, utc_now)
    if rates is None or len(rates) == 0:
        raise RuntimeError(f"No data retrieved for {symbol}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df

def compute_indicators(df):
    df['MA20'] = df['close'].rolling(20).mean()
    df['MA50'] = df['close'].rolling(50).mean()
    df['STD'] = df['close'].rolling(20).std()
    df['Upper_Band'] = df['MA20'] + (df['STD'] * 2.5)
    df['Lower_Band'] = df['MA20'] - (df['STD'] * 2.5)
    df['pctB'] = (df['close'] - df['Lower_Band']) / (df['Upper_Band'] - df['Lower_Band'])

    delta = df['close'].diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    roll_up = up.rolling(14).mean()
    roll_down = down.rolling(14).mean()
    RS = roll_up / roll_down
    df['RSI'] = 100.0 - (100.0 / (1.0 + RS))

    exp1 = df['close'].ewm(span=12, adjust=False).mean()
    exp2 = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

    df['Momentum'] = df['close'] - df['close'].shift(10)
    df['TR'] = df[['high','close']].max(axis=1) - df[['low','close']].min(axis=1)
    df['ATR'] = df['TR'].rolling(14).mean()

    df.dropna(inplace=True)
    return df

def generate_signals(df):
    # Simple backtest logic: Buy/Sell signals based on predicted vs RSI thresholds
    df['Predicted_Change'] = df['close'].pct_change().shift(-1)  # naive predictor: next close % change
    rsi_buy = df['RSI'].quantile(0.4)
    rsi_sell = df['RSI'].quantile(0.6)
    pred_buy = df['Predicted_Change'].quantile(0.6)
    pred_sell = df['Predicted_Change'].quantile(0.4)
    df['Signal'] = 0
    df.loc[(df['Predicted_Change'] > pred_buy) & (df['RSI'] < rsi_buy), 'Signal'] = 1
    df.loc[(df['Predicted_Change'] < pred_sell) & (df['RSI'] > rsi_sell), 'Signal'] = -1
    return df

def backtest(df, initial_capital=500.0, transaction_cost=0.0005, stop_loss_pct=0.1, take_profit_pct=0.2):
    cash = initial_capital
    holdings = 0
    entry_price = None
    positions = []
    portfolio_value = []

    for idx, row in df.iterrows():
        price = row['close']
        signal = row['Signal']

        # Enter long
        if signal == 1 and cash > 0:
            amount = cash * 0.5 * (1 - transaction_cost)
            holdings += amount / price
            cash -= amount
            entry_price = price
            positions.append({'Date': str(idx), 'Position': 'Buy', 'Price': price})

        # Exit long
        elif signal == -1 and holdings > 0:
            cash += holdings * price * (1 - transaction_cost)
            holdings = 0
            entry_price = None
            positions.append({'Date': str(idx), 'Position': 'Sell', 'Price': price})

        # Stop loss / take profit
        elif holdings > 0 and entry_price:
            if price <= entry_price * (1 - stop_loss_pct):
                cash += holdings * price * (1 - transaction_cost)
                holdings = 0
                entry_price = None
                positions.append({'Date': str(idx), 'Position': 'Stop Loss', 'Price': price})
            elif price >= entry_price * (1 + take_profit_pct):
                cash += holdings * price * (1 - transaction_cost)
                holdings = 0
                entry_price = None
                positions.append({'Date': str(idx), 'Position': 'Take Profit', 'Price': price})

        total_val = cash + holdings * price
        portfolio_value.append(total_val)

    df['Portfolio_Value'] = portfolio_value
    df['Daily_Return'] = df['Portfolio_Value'].pct_change()
    df['Cumulative_Return'] = (1 + df['Daily_Return'].fillna(0)).cumprod()
    total_return = (df['Portfolio_Value'].iloc[-1] - initial_capital) / initial_capital * 100
    return df, positions, total_return

# ----------------------------
# MAIN EXECUTION
# ----------------------------
if __name__ == "__main__":
    initialize_mt5()
    df = fetch_data(SYMBOL, TIMEFRAME, LOOKBACK_DAYS)
    df = compute_indicators(df)
    df = generate_signals(df)
    df, positions, total_return = backtest(df)

    print("Backtest complete!")
    print(f"Total return: {total_return:.2f}%")
    print("Positions executed:")
    for pos in positions[-5:]:  # last 5 positions
        print(pos)

    mt5.shutdown()
